In [1]:
import random
import requests
import pandas as pd
import math
import numpy as np
type(np.int32(np.atleast_1d(np.int32(0))))
from operator import attrgetter
import matplotlib.pyplot as plt
print("Ok")

Ok


In [16]:
"""応答関数読み込む"""
a = pd.read_csv("20knot_wide_ver2.csv", encoding="shift_jis")#widly_20_Sym:shift_jis
strip =a.values
print("Ok")

Ok


In [17]:
f=open("2峰応答データ.csv","w")

In [18]:
"""諸々の定義"""
n_fruid  = 1    #速度６種類
n_angle  = 36   #0~350まで36パターン 0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,
n_lambda = 80   #波長40パターン
Iy = 2661319053 #mm^2*m^2
y_y = 17.612 #[m] 中立軸
Zc=66.122*10**6
kissui=15.7

#freq_JWA=2*np.pi*np.array([0.0445953,0.0486315,0.053033,0.0578329,0.0630672,0.0687753,0.075,0.0817881,
               #0.0891906,0.097263,0.106066,0.1156658,0.1261345,0.1375506,0.15,0.1635762,0.1783811,
               #0.194526,0.2121321,0.2313317,0.252269,0.2751013,0.3000001,0.3271524,0.3567623])
omegalist=[]
for i in range(n_lambda):
    omegalist.append(strip[i][3])
omegalist=np.array(omegalist)#freq_JWA[::-1]
encount_omega_2 = np.arange( 0, 2.11,0.03 )
print(omegalist)

[2.57 2.54 2.51 2.48 2.45 2.42 2.39 2.36 2.33 2.3  2.27 2.24 2.21 2.18
 2.15 2.12 2.09 2.06 2.03 2.   1.97 1.94 1.91 1.88 1.85 1.82 1.79 1.76
 1.73 1.7  1.67 1.64 1.61 1.58 1.55 1.52 1.49 1.46 1.43 1.4  1.37 1.34
 1.31 1.28 1.25 1.22 1.19 1.16 1.13 1.1  1.07 1.04 1.01 0.98 0.95 0.92
 0.89 0.86 0.83 0.8  0.77 0.74 0.71 0.68 0.65 0.62 0.59 0.56 0.53 0.5
 0.47 0.44 0.41 0.38 0.35 0.32 0.29 0.26 0.23 0.2 ]


In [19]:
class Wave_Spectrum:    
    def __init__(self):
        self.Hv_1    =Hv_1
        self.theta_1 =theta_1
        self.lambda_1=lambda_1
        self.omega_1 =omega_1
        self.S_max1  =S_max1
        #self.s_1     =0
        self.Hv_2    =Hv_2
        self.theta_2 =theta_2
        self.lambda_2=lambda_2
        self.omega_2 =omega_2
        self.S_max2  =S_max2
        #self.s_2     =0
    
    def def_S_max(self,H,omega_m):
        HL=H/(1.56*4*math.pi**2/omega_m**2)
        if HL<0.026:
            log_10=0.4444*math.log10(HL)+0.5849
            a=10**(log_10)
            S_max=12.99*a**(-2.5)
        elif HL>=0.026:
            log10_gF=(0.1507-math.sqrt(0.1507**2+4*0.005882*0.8789+4*0.005882*math.log10(HL)))/(2*0.005882)
            gF=10**log10_gF
            gT=1.37*(1-1/(1+0.008*gF**(1/3))**5)
            S_max=12.99*gT**2.5
            
        return S_max if S_max <=84 else 84
        
    def spectrum(self,Hv,theta_m,lambda_m,omega_m,s,theta_func,omega):
        spectrum_11=0.250*((4.0*lambda_m+1.0)/4.0*omega_m**4.0)**lambda_m
        spectrum_12=math.gamma(lambda_m)
        spectrum_16=Hv**2.0
        spectrum_13=omega**(4.0*lambda_m+1.0)
        spectrum_14=2.0**(2.0*s-1.0)*(math.gamma(s+1.0))**2.0/math.pi/math.gamma(2.0*s+1.0)
        spectrum_15=abs((math.cos((theta_func-theta_m)/2.0*math.pi/180.0)))**(2.0*s)*math.exp(-(4.0*lambda_m+1.0)/4.0*(omega_m/omega)**4.0)
        return spectrum_11/spectrum_13*spectrum_14*spectrum_15*spectrum_16/spectrum_12
    
    def spectrum_by_angle(self,omega1,theta_func):#,omega,encount_angle
        """1峰目"""
        s_1= self.S_max1*(omega1/self.omega_1)**5 if omega1<=self.omega_1 else self.S_max1*(omega1/self.omega_1)**(-2.5)
        func1=self.spectrum(self.Hv_1,self.theta_1,self.lambda_1,self.omega_1,s_1,theta_func,omega1)
        """2峰目"""
        s_2= self.S_max2*(omega1/self.omega_2)**5 if omega1<=self.omega_2 else self.S_max2*(omega1/self.omega_2)**(-2.5)
        func2=self.spectrum(self.Hv_2,self.theta_2,self.lambda_2,self.omega_2,s_2,theta_func,omega1) 
        return (func1+func2)*int(360/(n_angle))*math.pi/180.0      #5°ずつだから


class Data_Handling(Wave_Spectrum):
    def __init__(self,column1,column2):
        self.column1=column1
        self.column2=column2
        self.response1=strip[:][self.column1]
        self.response2=strip[:][self.column2]
        self.Se_data=[]
        self.S_data=[]
        self.z1= 1000*9.8065/Zc if column1 == 17 else 1
        self.z2= 1000*9.8065/Zc if column1 == 17 else 1
        self.Matrix=[[0 for k in range(2)] for l in range(len(encount_omega_2))] #list(np.linspace(0, 0, len(encount_omega_2)))
        #self.omega_vertex=
        self.V  =Fn*math.sqrt(349.2*9.80655)
    
    def Response_calculation(self,line1):
        X_Conjugate = strip[line1][self.column1]*(math.cos(-strip[line1][self.column1+1]*math.pi/180)+1j*math.sin(-strip[line1][self.column1+1]*math.pi/180))
        Y= strip[line1][self.column2]*(math.cos(strip[line1][self.column2+1]*math.pi/180)+1j*math.sin(strip[line1][self.column2+1]*math.pi/180)) 
        return X_Conjugate*Y #strip[line1][self.column1]*strip[line1][self.column1]#
    
    def dw_function(self,omega_number):
        return np.abs(1.0-2.0*self.V/9.80655*omegalist[omega_number]*math.cos(encount_angle*math.pi/180.0))        
    
    def To_trans_Se(self):#Eomega1,2=Eomega_fore,aft
        """線形補間"""   
        Sf_re,Sa_re=self.Response_calculation(line).real*wave_values[w],self.Response_calculation(line-1).real*wave_values[w-1]
        Sf_im,Sa_im=self.Response_calculation(line).imag*wave_values[w],self.Response_calculation(line-1).imag*wave_values[w-1]
        dw= omegalist[w]-omegalist[w-1]
        for w_e in list_omega_j:        
            if switch == 1: #頂点のx座標が挟まれる場合
                self.Matrix[w_e][0] += (Sa_re+Sf_re)*dw/2/(omega_vertex/2-min(Eomega1,Eomega2))
                self.Matrix[w_e][1] += (Sa_im+Sf_im)*dw/2/(omega_vertex/2-min(Eomega1,Eomega2))
            elif switch == 2: #x軸との交点が挟まれる場合
                self.Matrix[w_e][0] += (Sa_re+Sf_re)*dw/2/max(Eomega1,Eomega2)
                self.Matrix[w_e][1] += (Sa_im+Sf_im)*dw/2/max(Eomega1,Eomega2)
            else:
                dwe1 = self.dw_function(w-1)
                dwe2 = self.dw_function(w)
                self.Matrix[w_e][0] +=(Eomega2-encount_omega_2[w_e])/(Eomega2-Eomega1)*Sa_re/dwe1+(encount_omega_2[w_e]-Eomega1)/(Eomega2-Eomega1)*Sf_re/dwe2
                self.Matrix[w_e][1] +=(Eomega2-encount_omega_2[w_e])/(Eomega2-Eomega1)*Sa_im/dwe1+(encount_omega_2[w_e]-Eomega1)/(Eomega2-Eomega1)*Sf_im/dwe2
            
    def final_keisan(self):
        F =np.zeros(len(encount_omega_2))
        for w_e in range(len(encount_omega_2)):
            F[w_e] += math.sqrt(self.Matrix[w_e][0]**2+self.Matrix[w_e][1]**2)*self.z1*self.z2
        for i in range(len(encount_omega_2)):
            f.write(str('%03.6f' %(F[i]))+",")
    
    def X_final_keisan(self):
        for i in range(len(encount_omega_2)):
            f.write(str('%03.6f' %(self.Matrix[i][0]*self.z1*self.z2))+",")
        for i in range(len(encount_omega_2)):
            f.write(str('%03.6f' %(self.Matrix[i][1]*self.z1*self.z2))+",")
        


"""ここからはクラスによらないdef関数が続く"""
def get_parameter_from_ind():
    """波高"""
    Hv_1=0.5+13.5*random.uniform(0,1)
    Hv_2= (0.01*Hv_1**2.3+0.5) +((5*(0.9*Hv_1**2-0.44*Hv_1)**0.1)-(0.01*Hv_1**2.3+0.5))*random.uniform(0,1)
    Hs    =np.sqrt(Hv_1**2+Hv_2**2)
    
    """波周波数"""
    omega_1 = 2*np.pi/18*np.exp(-0.005*Hs*3.28084)+(2*np.pi/5*np.exp(-0.017*Hs*3.28084)-2*np.pi/18*np.exp(-0.005*Hs*3.28084))*random.uniform(0,1)
    w_lower = omega_1 if omega_1>=0.5 else 0.5
    omega_2 = w_lower+(2*np.pi/4*np.exp(-0.014*Hs*3.28084)-w_lower)*random.uniform(0,1)
        
    """波向"""
    theta_1  =random.random()*360
    theta_2  =random.random()*360
    
    """s"""
    S_max1=1+83*random.random()
    S_max2=1+83*random.random()
    
    """尖度"""
    lambda_1 =0.1+9.9*random.uniform(0,1)
    lambda_temp=lambda_1 if lambda_1<=6 else 6
    lambda_2 =0.1+(lambda_temp-0.1)*random.uniform(0,1)

    """
    omega_1 = 2*np.pi/14 + (1.2-2*np.pi/14)*random.random()
    Wm2_min =omega_1 if omega_1>=0.6 else 0.6
    omega_2 =Wm2_min+(2*np.pi/4-Wm2_min)*random.random()

    T1      =2*np.pi/omega_1
    T2      =2*np.pi/omega_2
    Hv_1    =strip_data[int(T1)-4][24]+(strip_data[int(T1)-4][25]-strip_data[int(T1)-4][24])*random.random()
    H2_max  =np.sqrt(15**2-Hv_1**2) if np.sqrt(Hv_1**2+strip_data[int(T2)-4][25]**2)>=15 else strip_data[int(T2)-4][25]
    H2_min  =strip_data[int(T2)-4][24]
    Hv_2    = H2_min+(H2_max-H2_min)*random.random()"""

    return Hv_1,theta_1,lambda_1,omega_1,Hv_2,theta_2,lambda_2,omega_2,S_max1,S_max2

def search_omega_i(i):
    """間に入る出会い波周波数のリスト作成"""
    Eomega1=np.abs(omegalist[i-1]*(1-Fn*math.sqrt(349.2*9.80655)/9.80655*omegalist[i-1]*math.cos(encount_angle*math.pi/180)))
    Eomega2=np.abs(omegalist[i]*(1-Fn*math.sqrt(349.2*9.80655)/9.80655*omegalist[i]*math.cos(encount_angle*math.pi/180)))
    Eomega_fore = max(Eomega1,Eomega2)
    Eomega_aft =  min(Eomega1,Eomega2)
    switch=0#0は通常，１は頂点が挟まれる，２はｘとの交点が挟まれる
    if (omega_vertex - omegalist[i-1])*(omega_vertex -omegalist[i]) <= 0.000: #頂点のx座標が挟まれる場合
        Eomega_fore = omega_vertex/2 #頂点のy座標はx座標の半分だから
        switch=1
    elif (omega_vertex*2 - omegalist[i-1])*(omega_vertex*2 -omegalist[i]) <= 0: #x軸との交点が挟まれる場合
        Eomega_aft = np.abs(0.0)
        switch=2
    return np.arange(len(encount_omega_2))[np.array( (encount_omega_2-Eomega_fore)*(encount_omega_2-Eomega_aft) <= 0 ).flatten()] ,Eomega1, Eomega2,switch


def output(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2):
    f.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(math.cos(theta_1*math.pi/180)))+","+str('%03.8f' %(math.sin(theta_1*math.pi/180)))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1))+","+str('%03.8f' %(s_1))+",")
    f.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(math.cos(theta_2*math.pi/180)))+","+str('%03.8f' %(math.sin(theta_2*math.pi/180)))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+","+str('%03.8f' %(s_2)))
    f.write('\n')
    

In [2]:
for SeaState in range(50001):#海象データ数分だけ繰り返す
    random.seed()
    """この海象のパラメータ"""
    Hv_1,theta_1,lambda_1,omega_1,Hv_2,theta_2,lambda_2,omega_2 ,S_max1,S_max2= get_parameter_from_ind()
    wave=Wave_Spectrum()
    #wave.S_max1=wave.def_S_max(Hv_1,omega_1)
    #wave.S_max2=wave.def_S_max(Hv_2,omega_2)
    
    for speed in range(1): #6フルード数ごと 0,5,10,15,20,25
        Fn    = 0.176#strip_data[n_lambda*n_angle*k][0]#フルード数
        """データを扱うためのクラス"""
        Pitch=Data_Handling(13,13)#13
        Roll =Data_Handling(11,11)
        Heave=Data_Handling( 9, 9)
        Load =Data_Handling(17,17)#zを乗算するところのif分で列番号直すの忘れずに！
        Pitch_Roll  =Data_Handling(13,11)
        Roll_Heave  =Data_Handling(11, 9)
        Heave_Pitch =Data_Handling( 9,13)
        Load_Roll   =Data_Handling(17,11)
        Load_Heave  =Data_Handling(17, 9)
        Load_Pitch  =Data_Handling(17,13)
        
        for j,encount_angle in enumerate(np.arange(0,360,360/n_angle)): #36の角度 
            
            wave_values=[wave.spectrum_by_angle(omega1,encount_angle ) for omega1 in omegalist]
            omega_vertex  = 9.80655/(2*Fn*math.sqrt(349.2*9.80655)*math.cos(encount_angle*math.pi/180)) #頂点のx座標
            
            for w in range(1,len(omegalist)): #25周波数ごと n_lambda
                line = w+n_lambda*j+n_lambda*n_angle*speed    #行           
                list_omega_j,Eomega1, Eomega2,switch = search_omega_i(w)
                """それぞれの応答で変換実行"""                    
                Pitch.To_trans_Se()
                Roll.To_trans_Se()
                Heave.To_trans_Se()
                Load.To_trans_Se()
                Pitch_Roll.To_trans_Se()
                Roll_Heave.To_trans_Se()
                Heave_Pitch.To_trans_Se()
                Load_Roll.To_trans_Se()
                Load_Heave.To_trans_Se()
                Load_Pitch.To_trans_Se()
        """最終アウトプット"""        
        Pitch.final_keisan()
        Roll.final_keisan()
        Heave.final_keisan()
        Load.final_keisan()
        Pitch_Roll.X_final_keisan()
        Roll_Heave.X_final_keisan()
        Heave_Pitch.X_final_keisan()
        Load_Roll.X_final_keisan()
        Load_Heave.X_final_keisan()
        Load_Pitch.X_final_keisan()
        output(lambda_1,theta_1,S_max1,omega_1, Hv_1,lambda_2,theta_2,S_max2,omega_2,Hv_2)
        print(SeaState+1)

f.close()
print("end")

NameError: name 'random' is not defined